# Projeto Final | Analytics Engineering
----
**Engenharia de Dados e Garantia de Qualidade no Conjunto de Dados do Airbnb no Rio de Janeiro**

# Etapa 1:

**Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**

   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

## Bibliotecas

In [1]:
from sqlalchemy import create_engine, text as sql_text
import pandas as pd
import datetime

import os

In [2]:
## Importar modulo de conexão e implementação do banco  
import modules.database_connection_os as db

### Chamada da biblioteca para usar o SQL

In [3]:
%load_ext sql

Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


## Informações do banco de dados

Estabelecer uma conexão com um banco de dados PostgreSQL

In [4]:
# Configurar conexão
engine = db.engine_db()

In [5]:
# Testar a conexão
try:
    with engine.connect():
        print("Conexão bem sucedida!")
except Exception as e:
    print("Erro ao conectar:", e)

Conexão bem sucedida!


Criar schemas do banco de dados PostgreSQL

In [10]:
db.criar_schemas()

O Schema 'bronze' ja existe.
O Schema 'silver' ja existe.
O Schema 'gold' ja existe.


## Camada Bronze - Ingestão e Análise dos Dados

In [6]:
path = './dados/'

### Criar tabelas no Banco de dados

In [12]:
## Tabela Listing

df_listings = pd.read_csv(path + 'listings.csv')

In [13]:
# Criar e salvar o DataFrame na tabela "listings" dentro do esquema "bronze"
db.criar_tabela_df('bronze', 'listings', df_listings,'replace')

Criando a tabela 'listings' no schema 'bronze'....
A tabela 'listings' foi criada no schema 'bronze', e os dados foram inseridos.


In [15]:
## Tabela Reviews

df_reviews = pd.read_csv(path + 'reviews.csv')

In [16]:
# Criar e salvar o DataFrame na tabela "reviews" dentro do esquema "bronze"
db.criar_tabela_df('bronze', 'reviews', df_reviews,'replace')

Criando a tabela 'reviews' no schema 'bronze'....
A tabela 'reviews' foi criada no schema 'bronze', e os dados foram inseridos.


In [19]:
## Tabela Calendar

df_calendar = pd.read_csv(path + 'calendar.csv')

In [20]:
# Criar e salvar o DataFrame na tabela "calendar" dentro do esquema "bronze"
db.criar_tabela_df('bronze', 'calendar', df_calendar,'replace')

Criando a tabela 'calendar' no schema 'bronze'....
A tabela 'calendar' foi criada no schema 'bronze', e os dados foram inseridos.


# Etapa 2:

**Data Clean - Camada Silver**

   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

## Camada Silver - Limpeza de Dados e Criação de colunas

### Leitura das tabelas bronze

In [7]:
query = """
SELECT * FROM bronze.listings
"""
df_silver_listings = pd.read_sql(sql=sql_text(query), con=engine.connect())

In [8]:
query = """
SELECT * FROM bronze.reviews
"""
df_silver_reviews = pd.read_sql(sql=sql_text(query), con=engine.connect())

In [9]:
query = """
SELECT * FROM bronze.calendar
"""
df_silver_calendar = pd.read_sql(sql=sql_text(query), con=engine.connect())

### Relatório com informações do perfil dos dados a partir da biblioteca "ydata_profiling"

In [30]:
def data_profiling(df, df_name, output_file=None):
    from ydata_profiling import ProfileReport

    title = "Profiling Report - " + df_name
    filename = output_file + df_name + "_report.html"

    # criar relatório
    profile = ProfileReport(df, title= title) 

    # salvar resultados em um arquivo
    profile.to_file(filename) 

In [31]:
data_profiling(df_silver_listings, 'listings', 'data_profiling/')
# data_profiling(df_silver_reviews, 'reviews', 'data_profiling/')
# data_profiling(df_silver_calendar, 'calendar', 'data_profiling/')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\luana\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profiling\model\pandas\summary_pandas.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.fillna(np.nan)
c:\Users\luana\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profiling\model\pandas\summary_pandas.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.fillna(np.nan)
c:\Users\luana\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profiling\model\pandas\summary_pandas.py:39: FutureWarning: Downcasting object 

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Identificar e lidar com valores ausentes, duplicatas e outliers

In [38]:
def process_dataframe(df, df_name):

    print('Analisando a tabela ' + df_name + '\n')

    # Remover colunas com valores ausentes

    cols_df = df.columns

    print("Removendo colunas que possuem 100% de valores faltantes ...")
    df_cln = df.dropna(axis=1, how='all')

    cols_df_cln = df_cln.columns
    cols_removed = list(set(cols_df) - set(cols_df_cln))

    print('Colunas removidas da tabela ', df_name, ': \n', cols_removed)


    # Remover campos duplicados
    df_len = len(df_cln)

    print("\nRemovendo dados duplicados ...")

    df_cln = df_cln.drop_duplicates()

    dup_lines = len(df_cln) - df_len

    print('Foram removidas ', dup_lines, 'linhas da tabela ' + df_name)


    # Remover colunas constantes (opcional)
    print("\nRemovendo colunas constantes ...")

    list_constant = [col for col in df_cln.columns if df_cln[col].nunique() == 1]
    df_cln = df_cln.drop(list_constant, axis=1)

    print('Colunas constantes removidas da tabela ' + df_name + ': \n', list_constant)

    return df_cln

In [44]:
def remove_outliers(df, cols_list):
    """
    Remover outliers de colunas específicas. Considera-se outliers em colunas numéricas 
    selecionadas com valor acima ou abaixo de 2 desvios padrão da média.

    Parameters
    ----------
    df: dataframe
        Dataframe processado.
        
    cols_list: list
        Lista de colunas para considerar a eliminação de outliers.

    Returns
    -------
    df: dataframe
        Dataframe sem outliers.

    """
    qtd_lines = len(df)

    df_aux = df.copy()
    for col in cols_list:
        low_limit = df_aux[col].quantile(.02) 
        high_limit = df_aux[col].quantile(.98) 

        df = df[(df[col]>low_limit) & (df[col]<high_limit)] 

    qtd_lines = qtd_lines - len(df)

    print("\nQuantidade de linhas (outliers) eliminadas: ", qtd_lines )

    return df

In [46]:
### Teste funções: process_dataframe e remove_outliers 
df_tst = df_silver_listings
df_name = 'listings'

df = process_dataframe(df_tst, df_name)

cols_list = ['review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

df = remove_outliers(df, cols_list)
df.head(2)

Analisando a tabela listings

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  listings : 
 ['description', 'bedrooms', 'license', 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated']

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela listings

Removendo colunas constantes ...
Colunas constantes removidas da tabela listings: 
 ['scrape_id', 'amenities']

Quantidade de linhas (outliers) eliminadas:  27265


,id,listing_url,last_scraped,source,name,neighborhood_overview,picture_url,host_id,host_url,host_name,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17878,https://www.airbnb.com/rooms/17878,2023-12-27,city scrape,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,This is the one of the bests spots in Rio. Bec...,https://a0.muscache.com/pictures/65320518/3069...,68997,https://www.airbnb.com/users/show/68997,Matthias,...,4.83,4.91,4.77,4.67,f,1,1,0,0,1.90
1,25026,https://www.airbnb.com/rooms/25026,2023-12-27,city scrape,Rental unit in Rio de Janeiro · ★4.72 · 1 bedr...,Copacabana is a lively neighborhood and the ap...,https://a0.muscache.com/pictures/a745aa21-b8dd...,102840,https://www.airbnb.com/users/show/102840,Viviane,...,4.81,4.92,4.84,4.60,f,1,1,0,0,1.67


In [68]:
## Aplicar processamento em todas as tabelas silver

# colunas para considerar a eliminação de outliers
cols_listings = ['review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']
cols_reviews = []
cols_calendar = []

# tabela listings
df_process = process_dataframe(df_silver_listings, 'listings')
df_silver_listings = remove_outliers(df_process, cols_listings)

# tabela reviews
df_process = process_dataframe(df_silver_reviews, 'reviews')
df_silver_reviews = remove_outliers(df_process, cols_reviews)

# tabela calendar
df_process = process_dataframe(df_silver_calendar, 'calendar')
df_silver_calendar = remove_outliers(df_process, cols_calendar)

listings
Analisando a tabela listings

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  listings : 
 ['description', 'bedrooms', 'license', 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated']

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela listings

Removendo colunas constantes ...
Colunas constantes removidas da tabela listings: 
 ['scrape_id', 'amenities']

Quantidade de linhas (outliers) eliminadas:  27265


In [54]:
## Aplicar processamento em todas as tabelas silver

# colunas para considerar a eliminação de outliers
# cols_listings = ['review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']
# cols_calendar = []
# cols_reviews = []

# dict_tables = {'listings': [df_silver_listings, cols_listings],
#                'calendar': [df_silver_calendar, cols_calendar],
#                'reviews': [df_silver_reviews, cols_reviews]             
#                }

# for df_name, df_cols_list in dict_tables.items():

#     df = df_cols_list[0]
#     cols_list = df_cols_list[1]

#     df_process = process_dataframe(df, df_name)
#     df = remove_outliers(df_process, cols_list)

Analisando a tabela listings

Removendo colunas que possuem 100% de valores faltantes ...
Colunas removidas da tabela  listings : 
 ['description', 'bedrooms', 'license', 'neighbourhood_group_cleansed', 'bathrooms', 'calendar_updated']

Removendo dados duplicados ...
Foram removidas  0 linhas da tabela listings

Removendo colunas constantes ...
Colunas constantes removidas da tabela listings: 
 ['scrape_id', 'amenities']

Quantidade de linhas (outliers) eliminadas:  27265


### Padronização dos nomes e tipos das colunas

> Definir o tipo das colunas que não serão de texto

### Limpeza textual dos campos (remoção de caracteres especiais e erros de digitação)

# Etapa 3:

**Data Quality - Camada Silver**

   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

#### Verificação da precisão dos dados
ALGUMAS POSSÍVEIS VERIFICAÇÕES:

- Os meses precisam estar no intervalo: 1 <= meses <= 12
- Os anos precisam estar no intervalo: 1900 <= ano <= 2016
```python
df_cln.loc[(df_cln['yearOfRegistration']<1900) | (df_cln['yearOfRegistration']>2016), 'yearOfRegistration'] = 1900
```

- Os preços precisam ser maiores que 0

# Etapa 4:

**Testes de Qualidade - Camada Silver**

   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

# Etapa 5:

**Transformação de Dados com dbt - Camada Silver**

   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

# Etapa 6:

**Armazenamento de Dados em PostgreSQL - Camada Silver**

   - Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
   - Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

# Etapa 7:

**Validação de Expectativas com Great Expectations - Camada Silver**

   - Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
   - Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

# Etapa 8:

**Transformação de Dados com dbt - Camada Gold**

   - Utilize o dbt para criar a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "gold" e automatize a execução das transformações.
   - Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.